In [1]:
import pandas as pd

# CSV-Datei laden
url = 'https://datahub.io/core/population-city/r/unsd-citypopulation-year-both.csv'
data = pd.read_csv(url)

# Anzahl der einzigartigen Städte ermitteln
unique_cities = data['City'].nunique()
print(f"Anzahl der einzigartigen Städte im Datensatz: {unique_cities}")

Anzahl der einzigartigen Städte im Datensatz: 5156


In [2]:
import os
import pandas as pd
import json
import requests
from meteostat import Daily, Stations
from datetime import datetime
from data_preparation import download_files, files, output_folder
from data_preparation import data_import, data_cleaning, weather_data, geo_data, convert_date, population_data


In [3]:
df=data_import()
df.head()

Spalte 'Specie' fehlt in airquality-covid19-cities.json
waqi-covid-2022Q1.csv enthält nach Duplikat-Entfernung keine Daten mehr.


,Date,Country,City,Specie,count,min,max,median,variance
0,2015-01-06,KR,Jeonju,co,124,0.1,12.3,4.5,55.74
1,2015-01-22,KR,Jeonju,co,116,4.5,10.0,6.7,16.09
2,2015-03-30,KR,Jeonju,co,118,1.2,11.2,5.6,35.98
3,2015-05-27,KR,Jeonju,co,93,2.3,5.6,3.4,6.54
4,2015-02-03,KR,Jeonju,co,133,4.5,13.4,7.8,39.24


In [4]:
df=data_cleaning(df)
df.head()

Keine Geodaten gefunden für: washington d.c.
✅ Wetterdaten gesammelt für 578 Städte


✅ Datei wurde gespeichert: ./data/population_data.csv
✅ Datei wurde gespeichert: ./data/cleaned_data.csv


,Year,Month,Day,Country,City,Latitude,Longitude,Population,Co,No2,...,So2,Dew,Humidity,Tavg,Tmin,Tmax,Prcp,Wdir,Wspd,Pres
0,2019,2,28,SA,Abha,18.21639,42.50528,5616633.0,NaN,NaN,...,NaN,6.0,45.0,18.8,15.8,24.1,NaN,183.0,21.9,1022.3
1,2019,3,1,SA,Abha,18.21639,42.50528,5616633.0,NaN,NaN,...,NaN,12.0,64.0,17.0,13.5,23.0,NaN,185.0,21.2,1021.9
2,2019,3,2,SA,Abha,18.21639,42.50528,5616633.0,NaN,0.0,...,0.0,9.0,63.0,16.7,10.8,23.6,NaN,186.0,15.2,1022.3
3,2019,3,3,SA,Abha,18.21639,42.50528,5616633.0,NaN,0.0,...,0.0,6.0,55.0,15.9,10.8,22.0,NaN,196.0,16.5,1023.4
4,2019,3,4,SA,Abha,18.21639,42.50528,5616633.0,NaN,0.0,...,0.0,3.0,58.0,16.1,9.0,22.5,NaN,NaN,11.2,1023.8


In [5]:
df.columns

Index(['Year', 'Month', 'Day', 'Country', 'City', 'Latitude', 'Longitude',
       'Population', 'Co', 'No2', 'O3', 'Pm10', 'Pm25', 'So2', 'Dew',
       'Humidity', 'Tavg', 'Tmin', 'Tmax', 'Prcp', 'Wdir', 'Wspd', 'Pres'],
      dtype='object')

In [ ]:
df = df.copy()

In [ ]:
df = df.drop(columns=['variance', 'min', 'max'], errors='ignore')

In [ ]:
df = df.groupby(["Date", "Country", "City", "Specie"], as_index=False).agg({"median": "mean"})  

In [ ]:
df = df.pivot(index=["Date", "Country", "City"], columns="Specie", values='median').reset_index()

In [ ]:
df["City"] = df["City"].str.lower().str.strip()
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

In [ ]:
df.sample(10)

In [ ]:
df = geo_data(df)


In [ ]:
df.sample(10)

In [ ]:
df = weather_data(df)
df.sample(5)

In [ ]:
df = convert_date(df)
df.sample(5)

In [ ]:
df['City'] = df['City'].str.capitalize()

In [ ]:
df = df.copy()
df.sample(3)

In [ ]:
df_population = pd.read_csv(
                    './data/population.csv',
                    sep=',', 
                    header=0,
                    engine='python',  # Hilft oft bei Parsing-Problemen
                    on_bad_lines='skip',  # Methode zum Überspringen von fehlerhaften Zeilen
                    encoding='utf-8')  # Falls Sonderzeichen vorhanden sind
df_population.sample(3)

In [ ]:
df_population = df_population[['City', 'Value', 'Year']]
df_population.rename(columns={'Value': 'Population'}, inplace=True)
df_population.dropna(inplace=True)
df_population['Population'] = df_population['Population'].astype(int)
df_population.sample(3)

In [ ]:
# Spaltennamen anpassen
df.columns = df.columns.str.capitalize()
df['Year'] = df['Year'].astype(int)
df_population['Year'] = df_population['Year'].astype(int)
df.sample(3)

In [ ]:
# Sortieren nach 'City' und 'Year'
df = df.sort_values(by=['City', 'Year']).reset_index(drop=True)
print(df.head(10))
df_population = df_population.sort_values(by=['City', 'Year']).reset_index(drop=True)
df_population.head(10)

In [ ]:
# # **Nochmal sicherstellen, dass die Sortierung stimmt**
# if not df['Year'].is_monotonic_increasing:
#     raise ValueError("❌ 'df' ist nicht korrekt sortiert! Überprüfe die Sortierung nach 'Year'.")
# if not df_population['Year'].is_monotonic_increasing:
#     raise ValueError("❌ 'df_population' ist nicht korrekt sortiert!")

In [ ]:
df_population.isna().sum()

In [ ]:
#!!!!
df = df.merge(df_population, on=['City', 'Year'], how='left')
df['Population'] = df.groupby('City')['Population'].fillna(method='ffill').fillna(method='bfill')

In [ ]:
print(df.sample(3))


In [ ]:
print(df.shape)
df['Population'].isna().sum()


In [ ]:


    # Merge mit dem nächstgelegenen Jahr
    df = pd.merge_asof(df, df_population, on='Year', by='City', direction='nearest')

    # Datei speichern
    output_path = './data/population_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")

In [ ]:
def data_cleaning(df):
    """Bereinigung der Daten
    - nicht benötigte Spalten löschen
    - Zusammenfassung der Daten nach Datum, Land, Stadt und Spezies, so dass nur ein Messwert je Species (Median) pro Tag/ Stadt verbleibt
    - Spalte Species aufteilen
    - df als csv speichern im Datenverzeichnis
    """
    df = df.copy()

    df = df.drop(columns=['variance', 'min', 'max'], errors='ignore')

    df = df.groupby(["Date", "Country", "City", "Specie"], as_index=False).agg({"median": "mean"})  

    df = df.pivot(index=["Date", "Country", "City"], columns="Specie", values='median').reset_index()

    df["City"] = df["City"].str.lower().str.strip()
    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

    df = geo_data(df)

    df = weather_data(df)

    df = convert_date(df)

    df['City'] = df['City'].str.capitalize()

    df = population_data(df)

    #spalten sortieren
    df = df[['Year', 'Month', 'Day', 'Country', 'City', 'Latitude', 'Longitude', 'Population', 'Co', 'No2', 'O3', 'Pm10', 'Pm25',
       'Pressure', 'So2', 'Temperature', 'Wind-gust', 'Wind-speed', 'Dew', 'Humidity', 'Tavg', 'Tmin', 'Tmax', 'Prcp', 'Wdir', 'Wspd', 'Pres',
        ]]

    #Redundante Wetter-Spalten löschen
    df = df.drop(columns=[['Precipitation', 'Pressure', 'Uvi', 'Wd']], errors='ignore')
    df = df.drop(columns=[['Temperature', 'Wind-gust', 'Wind-speed']], errors='ignore')

    #Spalten mit mehr als 90% NaNs löschen
    df = df.loc[:, df.isnull().mean() < 0.9]

    output_path = './data/cleaned_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")

    return df

In [ ]:
df = df.copy()
df = data_cleaning(df)

In [ ]:
df.head()

In [ ]:
df.columns


In [ ]:
df.dtypes

In [ ]:
# alle raus, wo mehr al 90% der daten fehlen (aqi, D, Mepaqi, Neph, Pm1, Pol, Precipittion, psi, Temperature	Uvi	Wd	Wind-gust	Wind-speed ) # type: ignore
# wetterdaten aus einer quelle
# spalten sortieren
# wo sind die pop daten

In [ ]:
# # Funktion für Übersicht über dtypes, missing values, unique values etc.
def overview(df):
    '''
     Erstelle einen Überblick über einige Eigenschaften der Spalten eines DataFrames.
     VARs
         df: Der zu betrachtende DataFrame
     RETURNS:
         None
     '''
    display(pd.DataFrame({'dtype': df.dtypes,
                           'total': df.count(),
                           'missing': df.isna().sum(),
                           'missing%': df.isna().mean()*100,
                           'n_uniques': df.nunique(),
                           'uniques%': df.nunique()/df.shape[0]*100,
                           'uniques': [df[col].unique() for col in df.columns]
                          }))
    #return overview(df)


In [ ]:
overview(df)

In [ ]:
df.columns

In [ ]:
df = df.loc[:, df.isnull().mean() < 0.9]
df.head()

In [ ]:
def convert_date(df):
    # Convert 'Date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Split 'Date' column into 'year', 'month' and 'day'
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day

    # Remove 'Date' column
    if 'Date' in df.columns:
        df.drop(columns=['Date'], inplace=True)
    
    return (df)

In [ ]:
df_=convert_date(df)
df_.head()

In [ ]:
def population_data(df):
    '''
    Fügt dem jeder Stadt Einwohner hinzu
    '''
    df = df.copy()
    
    df_population = pd.read_csv(
                    './data/population.csv',
                    sep=',', 
                    header=0,
                    engine='python',  # Hilft oft bei Parsing-Problemen
                    on_bad_lines='skip',  # Methode zum Überspringen von fehlerhaften Zeilen
                    encoding='utf-8')  # Falls Sonderzeichen vorhanden sind


    df_population = df_population[['City', 'Value', 'Year']]
    df_population.rename(columns={'Value': 'Population'}, inplace=True)
    df_population.dropna(inplace=True)
    df_population['Population'] = df_population['Population'].astype(int)

    # Sicherstellen, dass die Spaltennamen übereinstimmen
    df.columns = df.columns.str.capitalize()
    df['Year'] = df['Year'].astype(int)
    df_population['Year'] = df_population['Year'].astype(int)

    # Sortieren für merge_asof (wichtig!)
    df = df.sort_values(by=['City', 'Year'])
    df_population = df_population.sort_values(by=['City', 'Year'])

    # Merge mit nächstgelegenem Jahr
    df = pd.merge_asof(df, df_population, on='Year', by='City', direction='nearest')

    # Datei speichern
    output_path = './data/population_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")

    return df

In [ ]:
#ALTE DEF()
# def population_data(df):
#     '''
#     Fügt dem jeder Stadt Einwohner hinzu
#     '''
#     df = df.copy()
    
#     df_population = pd.read_csv(
#                     './data/population.csv',
#                     sep=',', 
#                     header=0,
#                     engine='python',  # Hilft oft bei Parsing-Problemen
#                     on_bad_lines='skip',  # Methode zum Überspringen von fehlerhaften Zeilen
#                     encoding='utf-8')  # Falls Sonderzeichen vorhanden sind

#     df_population = df_population[['City', 'Value', 'Year']]

#     df_population.rename(columns={'Value': 'Population'}, inplace=True)

#     df_population.dropna(inplace=True)
#     df_population['Population'] = df_population['Population'].astype(int)

#     # Merge der beiden DataFrames basierend auf der "City"-Spalte
#     df.columns = df.columns.str.capitalize()
#     df['Year'] = df['Year'].astype(int)
#     df_population['Year'] = df_population['Year'].astype(int)
#     df = pd.merge(df, df_population, on=['City', 'Year'], how='left')
    

#     output_path = './data/population_data.csv'
#     os.makedirs(os.path.dirname(output_path), exist_ok=True)
#     df_population.to_csv(output_path, index=False)
#     print(f"✅ Datei wurde gespeichert: {output_path}")

#     return(df)

In [ ]:
df_['City'] = df_['City'].str.capitalize()
df_.head()

In [ ]:
df_= population_data(df_)
print(df_.shape)
df_.head()

In [ ]:
df_population = population_data(df)
df_population.head()

In [ ]:
df.columns = df.columns.str.capitalize()
df['Year'] = df['Year'].astype(int)
df_population['Year'] = df_population['Year'].astype(int)
df = pd.merge(df, df_population, on=['City', 'Year'], how='left')


In [ ]:
df.head()

In [ ]:
def convert_date(df):
    """
    Teilt die Spalte Date in Year, Month, Day auf
    """
    # Convert 'Date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Split 'Date' column into 'year', 'month' and 'day'
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day

    # Remove 'Date' column
    if 'Date' in df.columns:
        df.drop(columns=['Date'], inplace=True)

In [ ]:
df=data_cleaning(df)
df.head()

In [ ]:
df_population = pd.read_csv(
    '/Users/whypk/01Projekte/air_quality/data/population.csv',
    sep=',', 
    header=0,
    engine='python',  # Hilft oft bei Parsing-Problemen
    on_bad_lines='skip',  # Methode zum Überspringen von fehlerhaften Zeilen
    encoding='utf-8'  # Falls Sonderzeichen vorhanden sind
)

In [ ]:
print(df_population.head())
print(df_population.columns)
print(df_population.dtypes)
print(df_population.shape)

In [ ]:
df_population= df_population[['City', 'Value', 'Year']]
df_population.head()


In [ ]:

#df_population['population'] = df_population['population'].astype(int)
#df_population["City"] = df_population["City"].str.lower().str.strip()
df_population.rename(columns={'Value': 'Population'}, inplace=True)
df_population.head()

In [ ]:
df_population.dropna(inplace=True)
df_population['Population'] = df_population['Population'].astype(int)
df_population.head()

In [ ]:
df_ = pd.merge(df, df_population, on=['City', 'Year'], how='left')

In [ ]:
output_path = './data/population_data.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_population.to_csv(output_path, index=False)
print(f"✅ Datei wurde gespeichert: {output_path}")

In [ ]:
# Merge der beiden DataFrames basierend auf der "City"-Spalte
df_merged = pd.merge(df, df_population, on='City', how='left')

In [ ]:
# Funktion für Übersicht über dtypes, missing values, unique values etc.
def overview(df):
    '''
    Erstelle einen Überblick über einige Eigenschaften der Spalten eines DataFrames.
    VARs
        df: Der zu betrachtende DataFrame
    RETURNS:
        None
    '''
    display(pd.DataFrame({'dtype': df.dtypes,
                          'total': df.count(),
                          'missing': df.isna().sum(),
                          'missing%': df.isna().mean()*100,
                          'n_uniques': df.nunique(),
                          'uniques%': df.nunique()/df.shape[0]*100,
                          'uniques': [df[col].unique() for col in df.columns]
                         }))
overview(df_population)

In [ ]:
df_filtered_population = df_population[df_population['City'].isin(df['City'])][['Country or Area','City', 'Year', 'City type', 'Value']]
df_filtered_population['Value'] = df_filtered_population['Value'].astype(int)
df_filtered_population.head()